Least Squares

In [2]:
# Import packages.
import time
import cvxpy as cp
import numpy as np


time_begin = time.time()

# Generate data.
m = 20
n = 15
np.random.seed(1)
A = np.random.randn(m, n)
b = np.random.randn(m)

# Define and solve the CVXPY problem.
x = cp.Variable(n)
cost = cp.sum_squares(A @ x - b)
prob = cp.Problem(cp.Minimize(cost))
prob.solve()

# Print result.
print("\nThe optimal value is", prob.value)
print("The optimal x is")
print(x.value)
print("The norm of the residual is ", cp.norm(A @ x - b, p=2).value)


time_end = time.time()

time = time_end - time_begin

print('time:', time)


The optimal value is 7.005909828287484
The optimal x is
[ 0.17492418 -0.38102551  0.34732251  0.0173098  -0.0845784  -0.08134019
  0.293119    0.27019762  0.17493179 -0.23953449  0.64097935 -0.41633637
  0.12799688  0.1063942  -0.32158411]
The norm of the residual is  2.646867928002356
time: 0.005004405975341797


Linear

In [3]:
# Import packages.
import cvxpy as cp
import numpy as np
import time
time_begin = time.time()
# Generate a random non-trivial linear program.
m = 15
n = 10
np.random.seed(1)
s0 = np.random.randn(m)
lamb0 = np.maximum(-s0, 0)
s0 = np.maximum(s0, 0)
x0 = np.random.randn(n)
A = np.random.randn(m, n)
b = A@x0 + s0
c = -A.T@lamb0

# Define and solve the CVXPY problem.
x = cp.Variable(n)
prob = cp.Problem(cp.Minimize(c.T@x),
                 [A@x <= b])
prob.solve()

# Print result.
print("\nThe optimal value is", prob.value)
print("A solution x is")
print(x.value)
print("A dual solution is")
print(prob.constraints[0].dual_value)

time_end = time.time()

time = time_end - time_begin

print('time:', time)


The optimal value is -15.220912605552895
A solution x is
[-1.10133381 -0.16360111 -0.89734939  0.03216603  0.6069123  -1.12687348
  1.12967856  0.88176638  0.49075229  0.8984822 ]
A dual solution is
[6.98804552e-10 6.11756416e-01 5.28171747e-01 1.07296862e+00
 3.93758801e-09 2.30153870e+00 4.25703972e-10 7.61206896e-01
 8.36905707e-09 2.49370377e-01 1.30187001e-09 2.06014070e+00
 3.22417207e-01 3.84054343e-01 1.59493616e-09]
time: 0.0050048828125


QP

In [4]:
# Import packages.
import cvxpy as cp
import numpy as np
import time
time_begin = time.time()
# Generate a random non-trivial quadratic program.
m = 15
n = 10
p = 5
np.random.seed(1)
P = np.random.randn(n, n)
P = P.T@P
q = np.random.randn(n)
G = np.random.randn(m, n)
h = G@np.random.randn(n)
A = np.random.randn(p, n)
b = np.random.randn(p)

# Define and solve the CVXPY problem.
x = cp.Variable(n)
prob = cp.Problem(cp.Minimize((1/2)*cp.quad_form(x, P) + q.T@x),
                 [G@x <= h,
                  A@x == b])
prob.solve()

# Print result.
print("\nThe optimal value is", prob.value)
print("A solution x is")
print(x.value)
print("A dual solution corresponding to the inequality constraints is")
print(prob.constraints[0].dual_value)
time_end = time.time()

time = time_end - time_begin

print('time:', time)


The optimal value is 86.89141585569921
A solution x is
[-1.68244521  0.29769913 -2.38772183 -2.79986015  1.18270433 -0.20911897
 -4.50993526  3.76683701 -0.45770675 -3.78589638]
A dual solution corresponding to the inequality constraints is
[ 0.          0.          0.          0.          0.         10.45538054
  0.          0.          0.         39.67365045  0.          0.
  0.         20.79927156  6.54115873]
time: 0.01901412010192871


SOCP

In [5]:
# Import packages.
import cvxpy as cp
import numpy as np
import time
time_begin = time.time()
# Generate a random feasible SOCP.
m = 3
n = 10
p = 5
n_i = 5
np.random.seed(2)
f = np.random.randn(n)
A = []
b = []
c = []
d = []
x0 = np.random.randn(n)
for i in range(m):
    A.append(np.random.randn(n_i, n))
    b.append(np.random.randn(n_i))
    c.append(np.random.randn(n))
    d.append(np.linalg.norm(A[i]@x0 + b, 2) - c[i].T@x0)
F = np.random.randn(p, n)
g = F@x0

# Define and solve the CVXPY problem.
x = cp.Variable(n)
# We use cp.SOC(t, x) to create the SOC constraint ||x||_2 <= t.
soc_constraints = [
    cp.SOC(c[i].T@x + d[i], A[i]@x + b[i]) for i in range(m)
]
prob = cp.Problem(cp.Minimize(f.T@x),
                  soc_constraints + [F@x == g])
prob.solve()

# Print result.
print("The optimal value is", prob.value)
print("A solution x is")
print(x.value)
for i in range(m):
    print("SOC constraint %i dual variable solution" % i)
    print(soc_constraints[i].dual_value)
time_end = time.time()

time = time_end - time_begin

print('time:', time)

The optimal value is -9.582695716265807
A solution x is
[ 1.40303325  2.4194569   1.69146656 -0.26922215  1.30825472 -0.70834842
  0.19313706  1.64153496  0.47698583  0.66581033]
SOC constraint 0 dual variable solution
[array([0.61662526]), array([[ 0.35370661],
       [-0.02327185],
       [ 0.04253095],
       [ 0.06243588],
       [ 0.49886837]])]
SOC constraint 1 dual variable solution
[array([0.35283078]), array([[-0.14301082],
       [ 0.16539699],
       [-0.22027817],
       [ 0.15440264],
       [ 0.06571645]])]
SOC constraint 2 dual variable solution
[array([0.86510445]), array([[-0.114638  ],
       [-0.449291  ],
       [ 0.37810251],
       [-0.6144058 ],
       [-0.11377797]])]
time: 0.011008977890014648
